In [2]:
import threading

# Function to check if a number is prime
def is_prime(n):
    if n <= 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n**0.5)+1, 2):
        if n % i == 0:
            return False
    return True

# Thread worker function
def find_primes_in_range(start, end, result_list):
    for num in range(start, end):
        if is_prime(num):
            result_list.append(num)

# Main function to divide range and start threads
def find_primes_multithreaded(start, end, num_threads=4):
    threads = []
    result = []
    result_lock = threading.Lock()  # to avoid race conditions

    # Shared safe list appender
    def thread_task(start, end):
        local_primes = []
        find_primes_in_range(start, end, local_primes)
        with result_lock:
            result.extend(local_primes)

    # Divide range among threads
    step = (end - start) // num_threads
    for i in range(num_threads):
        range_start = start + i * step
        range_end = start + (i + 1) * step if i != num_threads - 1 else end
        t = threading.Thread(target=thread_task, args=(range_start, range_end))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    result.sort()
    return result

# Example usage
if __name__ == "__main__":
    start = 1
    end = 100
    primes = find_primes_multithreaded(start, end, num_threads=4)
    print("Prime numbers between", start, "and", end, "are:")
    print(primes)


Prime numbers between 1 and 100 are:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [3]:
import threading
from collections import Counter
import os

def count_words(lines, counter, lock):
    local_counter = Counter()
    for line in lines:
        words = line.strip().lower().split()
        local_counter.update(words)
    with lock:
        counter.update(local_counter)

def threaded_word_count(file_path, num_threads=4):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    total_lines = len(lines)
    chunk_size = total_lines // num_threads
    threads = []
    counter = Counter()
    lock = threading.Lock()

    for i in range(num_threads):
        start = i * chunk_size
        end = None if i == num_threads - 1 else (i + 1) * chunk_size
        thread_lines = lines[start:end]
        t = threading.Thread(target=count_words, args=(thread_lines, counter, lock))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    return counter

if __name__ == "__main__":
    file_path = 'large_text.txt'  
    if os.path.exists(file_path):
        word_counts = threaded_word_count(file_path, num_threads=4)
        for word, count in word_counts.most_common(10):
            print(f"{word}: {count}")
    else:
        print(f"File not found: {file_path}")


File not found: large_text.txt
